# import NER dataset

In [1]:
import pandas as pd

In [2]:
DF = pd.read_excel( r'E:\dataset\NER_total.xls' )
DF.sample(10)

,Sentence,Output
320,"['Strong', 'coding', 'skills', 'in', 'Python',...","['O', 'O', 'O', 'O', 'B', 'O', 'B', 'O', 'O', ..."
260,"['Role', ':', 'Data', 'Scientist']","['O', 'O', 'B', 'I']"
605,"['5', '+', 'years', 'of', 'SAS', 'experience']","['O', 'O', 'O', 'O', 'B', 'O']"
25,"['Programming', 'experience', 'in', 'R', 'is',...","['O', 'O', 'O', 'B', 'O', 'O', 'O']"
57,"['Data', 'visualization', 'experience', '(', '...","['B', 'I', 'I', 'O', 'O', 'O', 'O', 'O', 'B', ..."
689,"['.', 'work', 'experience', 'microsoft', 'cert...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
742,"['Software', 'programming', 'proficiency', 'wi...","['O', 'O', 'O', 'O', 'B', 'O', 'B', 'O', 'B', ..."
804,"['Familiarity', 'with', 'Git', 'and', 'Linux',...","['O', 'O', 'B', 'O', 'B', 'O', 'O', 'O', 'O']"
502,"['.', '(', '1', 'year', ')', ',', 'c++', '.net...","['O', 'O', 'O', 'O', 'O', 'O', 'B', 'B', 'O', ..."
47,"['Neural', 'networks', 'and', 'deep', 'learnin...","['B', 'I', 'O', 'B', 'I', 'O']"


# sample data

In [3]:
import random

ranIndex = random.randint( 0,len(DF) )

print('Sentence:')
print( DF.iloc[ranIndex]['Sentence'] )
print('Label')
print( DF.iloc[ranIndex]['Output'] )

Sentence:
['.', 'position', ':', 'intern', 'experience', ':', '1', 'year', 'exposure', ':', '•', 'module', 'coding', 'and', 'verification', 'using', 'verilog', 'hdl', '•', 'worked', 'on', 'linux', 'o.s.', '•', 'understanding', 'of', 'spi', ',', 'i2c', 'protocols', '•', 'compilation', 'using', 'altera', 'quartus', '•', 'simulation', 'using', 'modelsim', '•', 'report', 'preparation', 'and', 'documentation', 'education', 'little', 'rock', 'indian', 'school', '2007', 'master', 'of', 'science', 'in', 'technology', 'in', 'technology', 'school', 'of', 'information', 'sciences', 'vlsi', 'design', 'manipal', 'academy', 'of', 'higher', 'education', 'bachelor', 'of', 'engineering', 'in', 'engineering', 'srinivas', 'institute', 'of', 'technology', '-', 'mangalore', ',', 'karnataka', 'electronics', 'and', 'communication', 'visvesvaraya', 'technological', 'university', 'vidyodaya', 'p.u.', 'college', '-', 'udipi', ',', 'karnataka', 'skills', 'coding', '(', 'less', 'than', '1', 'year', ')', ',', 'hdl

In [4]:
sentence_list = DF['Sentence'].apply(eval)
label_list = DF['Output'].apply(eval)

sentence_list = [ w for w in sentence_list if len(w) <= 120]
label_list = [ w for w in label_list if len(w) <= 120]

# dataset split

In [5]:
import numpy as np
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split( sentence_list, label_list, test_size=0.2, random_state=42)

# PreProcess

In [6]:
import tensorflow as tf
from pprint import pprint

In [7]:
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import *
from keras_contrib.layers import CRF
from keras_contrib.losses import crf_loss
from keras_contrib.metrics import crf_accuracy
from keras.preprocessing.sequence import pad_sequences
from keras_bert import load_trained_model_from_checkpoint
from keras_bert import Tokenizer
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [8]:
max_seq_length = 128

In [9]:
label = {}
_label = {}


label_path = "E://bert-model//tag_dict.txt"
f_label = open(label_path, 'r+', encoding='utf-8')
for line in f_label:
    content = line.strip().split()
    label[content[0].strip()] = content[1].strip()
    _label[content[1].strip()] = content[0].strip()
    
    
vocab = {}
vocab_path = r"E:\Project\uncased_L-2_H-128_A-2\vocab.txt"
with open(vocab_path, 'r+', encoding='utf-8') as f_vocab:
    for line in f_vocab.readlines():
        vocab[line.strip()] = len(vocab)

In [10]:
def PreProcessInputData( text ):
    word_labels = []
    seq_types = []

    for sequence in text:
        len_text = len(sequence)

        ###########################################
        temp_word_labels = []
        temp_word_labels.append( 101 )            
        for w in sequence:
            temp_word_labels.append( vocab.get(str(w).lower(),100) )
        temp_word_labels.append( 102 )

        ###########################################
        ###########################################
        temp_seq_types = [1] * len(temp_word_labels) +  [0] * ( max_seq_length - len( temp_word_labels ))
        temp_word_labels = temp_word_labels + [0] * ( max_seq_length - len( temp_word_labels ))

        word_labels.append( temp_word_labels )
        seq_types.append( temp_seq_types )

    return word_labels, seq_types

In [11]:
def PreProcessOutputData( text ):
    tags = []
    for line in text:
        tag = [0]
        for item in line:
            tag.append( int(label[item.strip()]) )
        tag.append(0)
        tags.append(tag)

    pad_tags = pad_sequences(tags, maxlen=max_seq_length, padding="post", truncating="post")
    result_tags = np.expand_dims(pad_tags, 2)
    return result_tags

# Data PreProcessing

In [12]:
train_sentence_list_word_labels ,train_sentence_list_seq_types = PreProcessInputData( X_train )
train_output_label_list = PreProcessOutputData( y_train )

In [13]:
test_sentence_list_word_labels ,test_sentence_list_seq_types = PreProcessInputData( X_test )
test_output_label_list = PreProcessOutputData( y_test )

# basic network structure define

In [16]:
network_struct_dict = {
        'is_embeding_trainable' : True,
        'layer_dict_List' : [ 
            {'layer_type':'Bidirectional_LSTM', 'lstmDim': 128 ,'activation':'tanh', 'dropout' :0.3},
            {'layer_type':'Dense', 'units': 32 ,'activation':'sigmoid'},
        ],
        'optimizer' : Adam(1e-4),
        'batch_size' : 64,
        'epochs': 1
    }

# define_Layer

In [17]:
def define_Layer( layer_dict, my_input ):
    
    if layer_dict['layer_type'] == 'Bidirectional_LSTM':
        return Bidirectional( LSTM( layer_dict['lstmDim'],
                                         return_sequences = True,
                                         activation = layer_dict['activation'],
                                         dropout = layer_dict['dropout'],
                                         recurrent_dropout = layer_dict['dropout']) )(my_input)
    
    elif layer_dict['layer_type'] == 'Dense':
        return Dense(units = layer_dict['units'] , activation = layer_dict['activation'] )(my_input)
    
    elif layer_dict['layer_type'] == 'Dropout':
        return Dropout(dropout = 0.4 )(my_input)

# define network structure

In [18]:
def Train_Model( network_struct_dict ):
    print( '■' * 120 )
    print('network struct:')
    pprint( network_struct_dict )
    
    #################################################################
    ## use BERT embedding layer
    model_path = r"E:/Project/uncased_L-2_H-128_A-2/"
    bert = load_trained_model_from_checkpoint(
        model_path + "bert_config.json",
        model_path + "bert_model.ckpt",
        seq_len = max_seq_length
        )

    #make bert layer trainable
    for layer in bert.layers:
        layer.trainable = network_struct_dict['is_embeding_trainable']
        
    x1 = Input(shape=(None,))
    x2 = Input(shape=(None,))
    bert_out = bert([x1, x2])
    
    my_output = bert_out
    #################################################################
    # use layer_dict_List to build model
    for layer_dict in network_struct_dict['layer_dict_List']:
        my_input  = my_output
        my_output = define_Layer( layer_dict, my_input )
    
    
    #################################################################
    ## CRF output
    crf_out = CRF(len(label), sparse_target=True)(my_output)
    model = Model([x1, x2], crf_out)
    model.summary()

    model.compile(
        optimizer=network_struct_dict['optimizer'],
        loss=crf_loss,
        metrics=[crf_accuracy]
    )
    
    #################################################################
    ## training
    model.fit(x=[train_sentence_list_word_labels, train_sentence_list_seq_types],
                       y = train_output_label_list,
                       batch_size = network_struct_dict['batch_size'],
                       epochs = network_struct_dict['epochs'],
                       validation_split=0.2,
                       verbose = 1,
                       class_weight = 'auto')
    
    print('evaluate :')
    res = model.evaluate(x=[test_sentence_list_word_labels, test_sentence_list_seq_types],
                       y=test_output_label_list,batch_size=128)
    
    print( res )
    return res[1]

In [19]:
Train_Model(network_struct_dict)

network struct:
{'batch_size': 64,
 'epochs': 1,
 'is_embeding_trainable': True,
 'layer_dict_List': [{'activation': 'tanh',
                      'dropout': 0.3,
                      'layer_type': 'Bidirectional_LSTM',
                      'lstmDim': 128},
                     {'activation': 'sigmoid',
                      'layer_type': 'Dense',
                      'units': 32}],
 'optimizer': <keras.optimizers.Adam object at 0x0000000019060CC0>}




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.







Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None)         0                                            
__________________________

0.4214320073002263

# try different network_struct and keep the best

In [22]:
network_struct_dict_List = [
    ## network_struct
    {
        'is_embeding_trainable' : True,
        'optimizer' : RMSprop(1e-4),
        'batch_size' : 128,
        'epochs': 10,
        
        'layer_dict_List' : [ 
            {'layer_type':'Bidirectional_LSTM', 'lstmDim': 128 ,'activation':'tanh', 'dropout' :0.2},
            {'layer_type':'Bidirectional_LSTM', 'lstmDim': 64 ,'activation':'sigmoid', 'dropout' :0.2},
            {'layer_type':'Bidirectional_LSTM', 'lstmDim': 32 ,'activation':'sigmoid', 'dropout' :0.2},
        ]
    },
    
    ## network_struct
    {
        'is_embeding_trainable' : True,
        'optimizer' : RMSprop(1e-4),
        'batch_size' : 128,
        'epochs': 10,
        
        'layer_dict_List' : [ 
            {'layer_type':'Bidirectional_LSTM', 'lstmDim': 128 ,'activation':'tanh', 'dropout' :0.2},
            {'layer_type':'Bidirectional_LSTM', 'lstmDim': 64 ,'activation':'tanh', 'dropout' :0.2},
        ]
    },
    
    ## network_struct
    {
        'is_embeding_trainable' : True,
        'optimizer' : RMSprop(1e-4),
        'batch_size' : 64,
        'epochs': 10,
        'layer_dict_List' : [ 
            {'layer_type':'Bidirectional_LSTM', 'lstmDim': 128 ,'activation':'tanh', 'dropout' :0.3},
            {'layer_type':'Dense', 'units': 32 ,'activation':'sigmoid'},
        ]
    },
    
    ## network_struct
    {
        'is_embeding_trainable' : True,
        'optimizer' : Adam(1e-4),
        'batch_size' : 64,
        'epochs': 10,
        
        'layer_dict_List' : [ 
            {'layer_type':'Bidirectional_LSTM', 'lstmDim': 128 ,'activation':'tanh', 'dropout' :0.3},
            {'layer_type':'Dense', 'units': 32 ,'activation':'sigmoid'},
        ]
    }
]

In [23]:
best_network_struct = None
best_accurac = 0

for network_struct_dict in network_struct_dict_List:
    if best_accurac < Train_Model( network_struct_dict ):
        best_accurac = Train_Model( network_struct_dict )
        best_network_struct = network_struct_dict
        
pprint(best_network_struct)

network struct:
{'batch_size': 128,
 'epochs': 5,
 'is_embeding_trainable': True,
 'layer_dict_List': [{'activation': 'tanh',
                      'dropout': 0.2,
                      'layer_type': 'Bidirectional_LSTM',
                      'lstmDim': 128},
                     {'activation': 'sigmoid',
                      'dropout': 0.2,
                      'layer_type': 'Bidirectional_LSTM',
                      'lstmDim': 64},
                     {'activation': 'sigmoid',
                      'dropout': 0.2,
                      'layer_type': 'Bidirectional_LSTM',
                      'lstmDim': 32}],
 'optimizer': <keras.optimizers.RMSprop object at 0x0000000027F15AC8>}
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, None)         0                                            
________

Train on 544 samples, validate on 136 samples
Epoch 1/5
544/544 [==============================] - 45s 82ms/step - loss: 18.1339 - crf_accuracy: 0.5479 - val_loss: 17.4017 - val_crf_accuracy: 0.8262
Epoch 2/5
544/544 [==============================] - 39s 72ms/step - loss: 17.7718 - crf_accuracy: 0.8128 - val_loss: 17.3352 - val_crf_accuracy: 0.8262
Epoch 3/5
544/544 [==============================] - 40s 73ms/step - loss: 17.7098 - crf_accuracy: 0.8153 - val_loss: 17.3049 - val_crf_accuracy: 0.8262
Epoch 4/5
544/544 [==============================] - 41s 75ms/step - loss: 17.6786 - crf_accuracy: 0.8154 - val_loss: 17.2863 - val_crf_accuracy: 0.8262
Epoch 5/5
544/544 [==============================] - 41s 75ms/step - loss: 17.6537 - crf_accuracy: 0.8151 - val_loss: 17.2801 - val_crf_accuracy: 0.8262
evaluate :
171/171 [==============================] - 3s 16ms/step
[18.419665074487877, 0.8085442506081877]
network struct:
{'batch_size': 128,
 'epochs': 5,
 'is_embeding_trainable': True,

Train on 544 samples, validate on 136 samples
Epoch 1/5
544/544 [==============================] - 67s 124ms/step - loss: 19.2032 - crf_accuracy: 8.8078e-04 - val_loss: 18.4013 - val_crf_accuracy: 0.1885
Epoch 2/5
544/544 [==============================] - 52s 96ms/step - loss: 18.6863 - crf_accuracy: 0.1910 - val_loss: 18.0024 - val_crf_accuracy: 0.3152
Epoch 3/5
544/544 [==============================] - 52s 96ms/step - loss: 18.3681 - crf_accuracy: 0.3119 - val_loss: 17.7691 - val_crf_accuracy: 0.7291
Epoch 4/5
544/544 [==============================] - 52s 96ms/step - loss: 18.1383 - crf_accuracy: 0.6109 - val_loss: 17.6203 - val_crf_accuracy: 0.7819
Epoch 5/5
544/544 [==============================] - 53s 97ms/step - loss: 17.9867 - crf_accuracy: 0.7670 - val_loss: 17.5259 - val_crf_accuracy: 0.8252
evaluate :
171/171 [==============================] - 3s 19ms/step
[18.62789983916701, 0.8080524000508046]
{'batch_size': 128,
 'epochs': 5,
 'is_embeding_trainable': True,
 'layer_dic